<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#@title Settings {display-mode: "form"}

#@markdown #**Regular settings**
import os
import timeit
import numpy as np

Input = PARSER()

# @markdown #### Job name to be used as a header and in the final folder as _{jobname}_pyLiBELa.zip_
jobname = "test" #@param {type:"string"}

# @markdown #### Properties of the energy grid used in pose calculation. _grid_dimension_ is the size of the 3D array and _grid_spacing_ its spacing.

grid_spacing = 0.4 #@param {type:"number"}

grid_dimension = 30.0 #@param {type:"number"}

# @markdown #### Size of the search box used in ligand overlay optimization
search_box = 12.0 #@param {type:"number"}

# @markdown #### Algorithms for optimizing ligand overlay and interaction energy.
overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]

energy_optimizer = "subplex" #@param ["subplex", "direct", "isres", "crs", "esch", "stogo", "mma", "none"]

#@markdown #**Advanced settings**

atom_limit = 10000 #@param {type:"number"}
min_tol = 1E-4 #@param {type:'number'}
min_delta = 1E-4 #@param {type:'number'}
dock_min_tol = 1E-4 #@param {type:'number'}
timeout = 10 #@param {type:"number"}
min_timeout = 10 #@param {type:'number'}
use_score_optimizer = False; #@param {type:'boolean'}


time: 1.02 ms (started: 2024-03-19 20:46:16 +00:00)


# Instructions
### If you want to use CUDA, a GPU computing API that increases the speed in certain operations, you need to check the ```use_cuda``` box and select the GPU runtime from Google Colab by following the steps in the menu:
### `Runtime > Change Runtime Type > T4 GPU`

In [1]:
#@title Installing pyLiBELa {display-mode: "form"}
#@markdown The instalation process requires it to reestart the Colab. Don't be afraid if it happens.

#Installing Dependencies
%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel


#Compiling pyLiBELa
use_cuda = False #@param {type:"boolean"}

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2


time: 3min 10s (started: 2024-03-19 20:04:08 +00:00)


In [28]:
#@title Importing pyLiBELa {display-mode: "form"}
try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')



import os
import timeit
import numpy as np

Input = PARSER()

Input.grid_spacing = grid_spacing
Input.overlay_optimizer = overlay_optimizer
Input.energy_optimizer = energy_optimizer
Input.atom_limit = atom_limit

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True

scoring_function = "0"
Input.dielectric_model = "r"
#Input.diel = 2.0;
Input.scoring_function = int(scoring_function)
Input.grid_prefix = '../grids/McGrid'
Input.solvation_alpha = 0.1
Input.solvation_beta = -0.005

# Optimization parameter:
Input.min_tol = min_tol
Input.min_delta = min_delta
Input.dock_min_tol = dock_min_tol
Input.timeout = timeout
Input.min_timeout = min_timeout

if (Input.scoring_function < 3):
    delta = 2.5
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

Input.ligand_energy_model = "MMFF94";
Input.atomic_model_ff = "AMBER";
Input.use_score_optimizer = use_score_optimizer


pyLiBELa is imported!
time: 9.57 ms (started: 2024-03-19 20:46:55 +00:00)


In [42]:
#@title Uploading files{display-mode: "form"}
def File_extension_manager(name):
  role_flag = False

  name_ext = name.split('.')
#  name_ext = name_ext[-1]

  if name_ext[-1] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2'

  elif name_ext[-2] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2.gz'

  else:
    print('Check your file extension!\nIt needs to be .mol2 or .mol2.gz.')
    name_ext_string =''

  return role_flag,name_ext_string




import ipywidgets as widgets
from IPython.display import display
from google.colab import files

%cd '/content/'
!mkdir -p 'uploads'

%cd 'uploads'
!rm -rf *

uploaded = files.upload() #becomes a dictionary, for some reason
up_list = list(uploaded.keys())


rec_toggle = widgets.ToggleButtons(description='Receptor:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

lig_toggle = widgets.ToggleButtons(description='Reference Ligand:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

display(rec_toggle)
display(lig_toggle)

def rec_changed(changes):
  rec_name = rec_toggle.value
  #print('receptor em',rec_name)

def lig_changed(changes):
  lig_name = lig_toggle.value
  #print('ligante em',lig_name)

rec_toggle.observe(rec_changed,'value')
lig_toggle.observe(lig_changed,'value')
#print(toggle.value)

/content
/content/uploads


Saving 1SB1.rec.clean.mol2.gz to 1SB1.rec.clean.mol2.gz
Saving 1SB1.lig.am1bcc.mol2.gz to 1SB1.lig.am1bcc.mol2.gz


ToggleButtons(description='Receptor:', options=('1SB1.rec.clean.mol2.gz', '1SB1.lig.am1bcc.mol2.gz'), style=To…

ToggleButtons(description='Reference Ligand:', options=('1SB1.rec.clean.mol2.gz', '1SB1.lig.am1bcc.mol2.gz'), …

time: 11.4 s (started: 2024-03-19 21:06:48 +00:00)


In [47]:
#@title Preview
#@markdown Receptor in white, reference ligand in green and searchbox in blue.
try:
  import py3Dmol
except:
  !pip install py3Dmol
  import py3Dmol
aminoacids = ['MET', 'THR', 'ASN', 'LYS', 'SER', 'ARG', 'VAL', 'ALA', 'ASP', 'GLU', 'GLY', 'PHE', 'LEU', 'TYR', 'CYS', 'TRP', 'PRO', 'HIS', 'GLN', 'ILE', 'ASH', 'CYX', 'GLH', 'HIE', 'HID', 'HIP','HOH']



rec_name = rec_toggle.value
rec_flag,rec_ext = File_extension_manager(rec_name)
print('Receptor in',rec_name)
#print(rec_ext)

print(' ')

lig_name = lig_toggle.value
lig_flag,lig_ext = File_extension_manager(lig_name)
print('Ligand in',lig_name)
#print(lig_ext)


if rec_flag:
  REC = Mol2(Input, rec_name)
  print('Receptor uploaded succesfully!')
  print("It has %4d atoms." % REC.N)
#  !mv $rec_name 'rec'$rec_ext
#  rec_name = 'rec'+rec_ext

if lig_flag:
  RefLig = Mol2(Input, lig_name)
  print('Ligand uploaded succesfully!')
  print("It has %d atoms." % RefLig.N)
#  !mv $lig_name 'lig'$lig_ext
#  lig_name = 'lig'+lig_ext


Coord = COORD_MC()
center = Coord.compute_com(RefLig)


%cd '/content/'

!rm -rf lig_ref.pdb rec.pdb search_box.pdb

!obabel -imol2 'uploads/'$rec_name -opdb -O lig_ref.pdb
!obabel -imol2 'uploads/'$lig_name -opdb -O rec.pdb

lig_ref_pdb = open('lig_ref.pdb', 'r').read()
rec_pdb = open('rec.pdb','r').read()


def join_pdb(name_file_rec,name_file_lig,name_file_merge):
  !cp $name_file_rec temp_rec.pdb
  !cp $name_file_lig temp_lig.pdb

  !sed -i '/END\|CONECT/d' temp_rec.pdb #tira o end e o conect do arquivo do receptor

  !sed -i 's/A  /B  /g' temp_lig.pdb #troca a cadeia A por B no arquivo do ligante

  !sed -i '/CONECT/d' temp_lig.pdb #tira o conect do arquivo do ligante

  !cat temp_rec.pdb temp_lig.pdb > $name_file_merge #concatena os arquivos

  !rm -rf temp_rec.pdb temp_lig.pdb


join_pdb('rec.pdb','lig_ref.pdb','complex1.pdb')
complex1_pdb = open('complex1.pdb','r').read()


xend,xbegin = center[0]+search_box/2, center[0]-search_box/2
yend,ybegin = center[1]+search_box/2, center[1]-search_box/2
zend,zbegin = center[2]+search_box/2, center[2]-search_box/2


Writer = WRITER(Input)
Writer.write_box(center, xbegin, ybegin, zbegin, xend, yend, zend)
!mv 'box.pdb' 'search_box.pdb'

search_box_pdb = open('search_box.pdb','r').read()
view = py3Dmol.view(width=800,height=600)

model=0

view.addModel(search_box_pdb,'pdb')
view.setStyle({'model':0},{'stick':{"color":"#4565EA","width":0.5}})

model+=1

view.addModel(lig_ref_pdb,'pdb')
view.setStyle({'model':model},{'stick': {'colorscheme':'greenCarbon'}})
model+=1


view.addModel(complex1_pdb,'pdb')
view.setStyle({'model':model,'chain':'B'},{'opacity':0})
view.setStyle({'model':model,'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
view.setStyle({ 'model':model,'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})


view.setBackgroundColor('white')

view.zoomTo()
view.show()

Receptor in 1SB1.rec.clean.mol2.gz
 
Ligand in 1SB1.lig.am1bcc.mol2.gz
Receptor uploaded succesfully!
It has    0 atoms.
Ligand uploaded succesfully!
It has 0 atoms.
/content
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is SMALL)

1 molecule converted
1 molecule converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

time: 1.19 s (started: 2024-03-19 21:08:29 +00:00)


In [8]:

#@title Reading search ligand {display-mode: "form"}

#Reading Ligand SMILES

%cd '/content/'
!rm -rf lig2.pdb lig2.smi

ligand_smiles = "N1([C@@H](CCC1)C(=O)N[C@@H]1CC2=C(N=C(S2)N)CC1)C(=O)[C@H](NS(=O)(=O)Cc1ccccc1)CC1CCCCC1" #@param {type:"string"}


Lig2 = Mol2()
lig2_flag = False
if Mol2.parse_smiles(Lig2, Input, ligand_smiles, 'Lig2'):
   print('Ligand SMILES parsed successfully!')
   print(ligand_smiles)
   lig2_flag = True

else:
  print('Oops, something wrong with you ligand... \nIt may be way too big or have an unknown atom.')

if lig2_flag:
  print('Search ligand has %4d atoms' % Lig2.N)

  f = open("lig2.smi", "w")
  f.write(ligand_smiles)
  f.close()

  !obabel lig2.smi --gen3d -opdb -O lig2.pdb

  lig2_pdb = open('lig2.pdb', 'r').read()

  view = py3Dmol.view()

  view.addModel(lig2_pdb,'pdb')
  view.setStyle({'model':0},{'stick': {'colorscheme':'yellowCarbon'}})

  view.setBackgroundColor('white')

  view.zoomTo()
  view.show()

/content
Ligand SMILES parsed successfully!
N1([C@@H](CCC1)C(=O)N[C@@H]1CC2=C(N=C(S2)N)CC1)C(=O)[C@H](NS(=O)(=O)Cc1ccccc1)CC1CCCCC1
Search ligand has   78 atoms
1 molecule converted


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [24]:
#@title Grid Generation
%load_ext autotime
!rm -rf McLiBELa_dock.mol2.gz
import timeit


Coord = COORD_MC()
HB = FindHB()
Ene  = Energy2(Input)

for i in range(len(REC.residue_pointer)-1):
  HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);

HB.find_ligandHB(lig_name, RefLig);
print('The receptor has %5d / %5d HB donors/acceptors around the active site.' % (len(REC.HBdonors), len(REC.HBacceptors)));

Dock = Docker(Writer)
center = Coord.compute_com(RefLig)

print()
start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);
print('Starting energy: %7.3f kcal/mol' % start_energy);
print()
print('Generating grids. This may take a while..')

start_time = timeit.default_timer()
Grids = Grid(Input, Writer, REC, center)
time = timeit.default_timer() - start_time

time_per_atom = time/REC.N
print('Grids computed, and it took %.2f s! That means %.4f s per atom.' %(time,time_per_atom))
grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);
print('Grid original energy: %7.3f kcal/mol' % grid_energy);
print('Grid error: %7.3f' % abs(100.*(start_energy-grid_energy)/start_energy));
print()
print()


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The receptor has    18 /    26 HB donors/acceptors around the active site.

Starting energy: -47.009 kcal/mol

Generating grids. This may take a while..
Grids computed, and it took 275.52 s! That means 0.0684 s per atom.
Grid original energy: -45.979 kcal/mol
Grid error:   2.192


time: 4min 35s (started: 2024-03-19 20:34:30 +00:00)


In [ ]:
#@title Docking and showing the result

print('Starting docking calculation...')
Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
print('Docking calculation finished!')

Writer.write_box(center, Grids.xbegin, Grids.ybegin, Grids.zbegin, Grids.xend, Grids.yend, Grids.zend)

!rm -rf lig2.pdb
#trocar aqui
!obabel -imol2 McLiBELa_dock.mol2.gz -opdb -O lig2.pdb

lig2_pdb = open('lig2.pdb', 'r').read()

box_pdb = open('box.pdb','r').read()

join_pdb('rec.pdb','lig2.pdb','complex2.pdb')
complex2_pdb = open('complex2.pdb','r').read()


!obrms lig.mol2.gz McLiBELa_dock.mol2.gz



#@title Docked complex
#@markdown Do you want to show the grid box?
show_grid = True #@param {type:"boolean"}
#@markdown Do you want to show the search box?
show_box = True #@param {type:"boolean"}
#@markdown Do you want to show the reference ligand? If selected, it will be shown in green.
show_ref = True #@param {type:"boolean"}
#@markdown Do you want to show the target protein?
show_rec = True #@param {type:"boolean"}
#@markdown Do you want to show the search ligand? If selected, it will be shown in yellow.
show_lig2 = True #@param {type:"boolean"}
#@markdown Do you want to show residues from the protein close to the search ligand?
show_close_res = True #@param {type:"boolean"}
#@markdown How close do you want them to be? Put the radius in Angstroms
dist=5 #@param {type:"integer"}


view = py3Dmol.view(width=800,height=600)

model=0

if show_grid:
  view.addModel(box_pdb,'pdb')
  view.setStyle({'model':model}, {'stick':{"color":"#ff0000","width":2}})
  #print(model)
  model+=1

if show_box:
  view.addModel(search_box_pdb,'pdb')
  view.setStyle({'model':model},{'stick':{"color":"#4565EA","width":0.5}})
  #print(model)
  model+=1

if show_ref:
  view.addModel(lig_ref_pdb,'pdb')
  view.setStyle({'model':model},{'stick': {'colorscheme':'greenCarbon','opacity':0.7}})
  model+=1

if show_rec:

  view.addModel(complex2_pdb,'pdb')
  view.setStyle({'model':model,'chain':'B'},{'opacity':0})
  view.setStyle({'model':model,'chain':'A'},  {"cartoon":{'arrows':True,'colorscheme':'ssPymol','opacity':0.7}})
  view.setStyle({ 'model':model,'chain':'A', 'not':{'resn':aminoacids}}, {'sphere':{}})

  if show_close_res:
    view.addStyle({'model':model,'chain':'A','within':{'distance':'%.d'%dist, 'sel':{'chain':'B'}},'byres':True}, {'stick': {'colorscheme':'skyblueCarbon'}})

  if show_lig2:
      view.setStyle({'model':model,'chain':'B'},{'stick': {'colorscheme':'yellowCarbon'}})
  model+=1

else:
  if show_lig2:
    view.addModel(lig2_pdb,'pdb')
    view.setStyle({'model':model},{'stick': {'colorscheme':'yellowCarbon'}})
    model+=1
  if show_close_res:
    print('To show the residues close to the ligand, you need to select the show_rec box.')


view.setBackgroundColor('white')

view.zoomTo()
view.show()

Starting docking calculation...
Docking calculation finished!
2 molecules converted
RMSD 182L_H.pdb:Lig2 inf


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

time: 48.1 s (started: 2024-03-18 20:01:45 +00:00)


In [ ]:
#@title Zipping and downloading files

%%capture
!mkdir pyLiBELa_docking
!cp -r *.pdb *.mol2 *.smi McLiBELa.log uploads/ pyLiBELa_docking

! tar -zcvf pyLiBELa_docking.tar.gz pyLiBELa_docking

files.download('pyLiBELa_docking.tar.gz')

mkdir: cannot create directory ‘pyLiBELa_docking’: File exists
pyLiBELa_docking/
pyLiBELa_docking/rec.pdb
pyLiBELa_docking/uploads/
pyLiBELa_docking/uploads/1SB1.rec.clean.mol2.gz
pyLiBELa_docking/uploads/lig.mol2.gz
pyLiBELa_docking/uploads/rec.mol2.gz
pyLiBELa_docking/lig2.pdb
pyLiBELa_docking/search_box.pdb
pyLiBELa_docking/lig2.mol2
pyLiBELa_docking/McLiBELa.log
pyLiBELa_docking/complex1.pdb
pyLiBELa_docking/lig2.smi
pyLiBELa_docking/box.pdb
pyLiBELa_docking/complex2.pdb
pyLiBELa_docking/lig_ref.pdb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 418 ms (started: 2024-03-18 20:07:32 +00:00)
